In [4]:
import angr
proj = angr.Project('/bin/true')

In [9]:
import IPython
py = IPython.get_ipython()
py.Completer.use_jedi = False

In [16]:
import monkeyhex
print(proj.arch)
print(proj.entry)
print(proj.filename)

<Arch AMD64 (LE)>
4204512
/bin/true


In [19]:
proj.loader


<Loaded true, maps [0x400000:0xa07fff]>

In [20]:
proj.loader.shared_objects

OrderedDict([('true', <ELF Object true, maps [0x400000:0x40a23f]>),
             ('libc.so.6', <ELF Object libc.so.6, maps [0x500000:0x6f5e77]>),
             ('ld-linux-x86-64.so.2',
              <ELF Object ld-linux-x86-64.so.2, maps [0x700000:0x73730f]>),
             ('extern-address space',
              <ExternObject Object cle##externs, maps [0x800000:0x87ffff]>),
             ('cle##tls',
              <ELFTLSObjectV2 Object cle##tls, maps [0x900000:0x91500f]>)])

In [21]:
proj.loader.min_addr

0x400000

In [22]:
proj.loader.max_addr

0xa07fff

In [24]:
proj.loader.main_object

<ELF Object true, maps [0x400000:0x40a23f]>

In [25]:
proj.loader.main_object.execstack

False

In [26]:
proj.loader.main_object.pic

True

In [33]:
"""
Se tiene project.factory.block() que sirve para extraer un bloque de código. Angr analiza codigo en unidades de bloques basicos
"""

block = proj.factory.block(proj.entry)
block.pp()
print("////")
block.instructions
block.instruction_addrs


        _start:
4027e0  xor     ebp, ebp
4027e2  mov     r9, rdx
4027e5  pop     rsi
4027e6  mov     rdx, rsp
4027e9  and     rsp, 0xfffffffffffffff0
4027ed  push    rax
4027ee  push    rsp
4027ef  xor     r8d, r8d
4027f2  xor     ecx, ecx
4027f4  lea     rdi, [0x402710]
4027fb  call    qword ptr [0x409fc0]
////


(0x4027e0,
 0x4027e2,
 0x4027e5,
 0x4027e6,
 0x4027e9,
 0x4027ed,
 0x4027ee,
 0x4027ef,
 0x4027f2,
 0x4027f4,
 0x4027fb)

In [37]:
"""
Tambien se puede usar un objeto bloque para obtener otra representacion de un bloque de codigo 
"""

block.capstone
block.vex

IRSB <0x21 bytes, 11 ins., <Arch AMD64 (LE)>> at 0x4027e0

<h1>States</h1>

In [40]:
"""
El objecto Project solo representa una imagen inicializada del programa.
Cuando se realiza una ejecución con angr, se esta trabajando con un objeto especifico
que esta representando un estado de programa simulado. Un SimState
"""

state = proj.factory.entry_state()
state

<SimState @ 0x4027e0>

In [56]:
""" 
Estos contienen la memoria, registros, datos de file system y cualquier "información viva"
de un programa que pueda ser cambiada por la ejecución.
"""

print(state.regs.rax)
print(state.regs.rax)
print(state.mem[proj.entry].int.resolved)

<BV64 0x1c>
<BV64 0x1c>
<BV32 0x8949ed31>


In [51]:
"""
Estos valores mostrados no son enteros de Python. estos son "bitvectors"
los integer de python no tienen la misma semantica que los words de un CPU.
hay que tener en cuenta esto para manejar información de CPU utilizando angr.

"""

import claripy
bv = claripy.BVV(0x1234, 32) #Se crea un bitvector, de 32-bit con valor 0x1234
print(bv)
print(bv.length) #Notar que los bitvector tienen el atributo length, que especifican de cuantos bits son.
state.solver.eval(bv) #Se convierte el valor a un entero de python.

<BV32 0x1234>
32


0x1234

In [60]:
state.regs.rsi = claripy.BVV(3, 64)
state.regs.rsi

state.mem[0x1000].long = 4
state.mem[0x1000].long.resolved

<BV64 0x4>

In [59]:
print(state.regs.rdi)

<BV64 0x1>


<h1>Simulation Managers</h1>

In [63]:
"""
Si un estado representa un programa en un punto dado de tiempo. 
Tiene queh aber un modo de conseguir el siguiente punto de tiempo de la ejecución.
Los manejadores de simulación son la interfaz primaria en angr para realizar la ejecución o simulación.
"""

simgr = proj.factory.simulation_manager(state)
print(simgr)
simgr.active

<SimulationManager with 1 active>


[<SimState @ 0x4027e0>]

In [64]:
""" 
Un manejador de simulación puede tener varios "stashes" de estados. el stash por defecto es active,
esto es inicializado. 
"""

simgr.step()

<SimulationManager with 1 active>

In [68]:
"""
Ahora tenemos un bloque basico con una ejecución simbolica. Podemos ver el stash activo y darnos cuenta 
de la actualización y más cosas. 
"""

print(simgr.active)
print(simgr.active[0].regs.rip)
print(state.regs.rip)

[<SimState @ 0x529ce0>]
<BV64 0x529ce0>
<BV64 0x4027e0>


<h1>Analyses</h1>

In [ ]:
""" 
Angr viene incluido con muchos analizadores que pueden ser sados para extraer información
de programas
"""

proj.analyses.            # Press TAB here in ipython to get an autocomplete-listing of everything:
 proj.analyses.BackwardSlice        proj.analyses.CongruencyCheck      proj.analyses.reload_analyses
 proj.analyses.BinaryOptimizer      proj.analyses.DDG                  proj.analyses.StaticHooker
 proj.analyses.BinDiff              proj.analyses.DFG                  proj.analyses.VariableRecovery
 proj.analyses.BoyScout             proj.analyses.Disassembly          proj.analyses.VariableRecoveryFast
 proj.analyses.CDG                  proj.analyses.GirlScout            proj.analyses.Veritesting
 proj.analyses.CFG                  proj.analyses.Identifier           proj.analyses.VFG
 proj.analyses.CFGEmulated          proj.analyses.LoopFinder           proj.analyses.VSA_DDG
 proj.analyses.CFGFast              proj.analyses.Reassembler


In [74]:
proj = angr.Project('/bin/true', auto_load_libs=False)
cfg = proj.analyses.CFGFast()
print(cfg)
print(cfg.graph)

entry_node = cfg.model.get_any_node(proj.entry)
len(list(cfg.graph.successors(entry_node)))

<CFGFast Analysis Result at 0x7f39200aa750>
DiGraph with 1361 nodes and 2169 edges


0x1